In [1]:

import numpy as np
import music21 as m21
import os

import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.layers import Activation

import sys, re
import random

DEBUGmode = True 


Using TensorFlow backend.


In [2]:

def init_gtrbass_streams():
    # initialize two parts and one stream
    
    
    stream1 = m21.stream.Stream()
    stream1.keySignature = m21.key.Key('C', 'major')
    stream1.timeSignature = m21.meter.TimeSignature('4/4')
     
    ## music21.tempo.MetronomeMark Quarter=165.0

#guitar
    gtr_part    = m21.stream.Part()
    gtr_part.id = 'Guitar Stream'
    gtr_part.insert(m21.instrument.ElectricGuitar())
    gtr_part.clef = m21.clef.TrebleClef()  
    
#Bass     
    bass_part    = m21.stream.Part()
    bass_part.id = 'Bass Guitar Stream'
    bass_part.insert(m21.instrument.ElectricBass())
    bass_part.clef = m21.clef.BassClef()  
    
    return (stream1, gtr_part, bass_part)





In [3]:

ramones_midi_data = pd.read_pickle('ramones_GtrBass_all_data_transposed')
ram_scores = ramones_midi_data.as_matrix()

unique_Ramones_midi_data = ramones_midi_data.drop_duplicates()

frequencies = ramones_midi_data.groupby(['Gtr0','Gtr1','Gtr2','Gtr3','Gtr4','Gtr5','Gtr_Mute', 'Gtr_Hold', 'Bass1', 'Bass_Mute', 'Bass_Hold']).size()
frequencies.sort_values(ascending=False, inplace=True)


ramones_midi_data_notesOnly = ramones_midi_data.loc[:,['Gtr0','Gtr1','Gtr2','Gtr3','Gtr4','Gtr5', 'Bass1']]
frequencies_notes = ramones_midi_data_notesOnly.groupby(['Gtr0','Gtr1','Gtr2','Gtr3','Gtr4','Gtr5', 'Bass1']).size()
frequencies_notes.sort_values(ascending=False, inplace=True)


ramones_midi_data_GtrOnly = ramones_midi_data.loc[:,['Gtr0','Gtr1','Gtr2','Gtr3','Gtr4','Gtr5']]
frequencies_notes_gtr = ramones_midi_data_GtrOnly.groupby(['Gtr0','Gtr1','Gtr2','Gtr3','Gtr4','Gtr5']).size()
frequencies_notes_gtr.sort_values(ascending=False, inplace=True)



ramones_midi_data['combined'] = list(zip(ramones_midi_data.Gtr0, ramones_midi_data.Gtr1,
         ramones_midi_data.Gtr2, ramones_midi_data.Gtr3,
         ramones_midi_data.Gtr4, ramones_midi_data.Gtr5, 
         ramones_midi_data.Gtr_Mute, ramones_midi_data.Gtr_Hold,
         ramones_midi_data.Bass1, ramones_midi_data.Bass_Hold,
         ramones_midi_data.Bass_Mute))


rmd_frequencies = ramones_midi_data.groupby(['combined']).size()
rmd_frequencies.sort_values(inplace=True,  ascending=False)



# make DF and add index: 
rmd_frequencies_DF = pd.DataFrame(rmd_frequencies)
rmd_frequencies_DF['frq'] = range(0, len(rmd_frequencies_DF))

rmd_frequencies_DF['my_index'] = rmd_frequencies_DF.index

ramones_midi_data = ramones_midi_data.merge(rmd_frequencies_DF, how='left',  left_on='combined', right_on='my_index' )

del ramones_midi_data[ 'my_index']

ramones_midi_data.columns = ['Gtr0',      'Gtr1',      'Gtr2',      'Gtr3',      'Gtr4',
            'Gtr5',  'Gtr_Mute',  'Gtr_Hold',     'Bass1', 'Bass_Mute',
       'Bass_Hold',  'combined',     'count',   'freq']



In [4]:

indices_word = dict(zip(ramones_midi_data.freq.values, ramones_midi_data.combined))
word_indices = dict(zip(ramones_midi_data.combined, ramones_midi_data.freq.values))


#chars = set(str1)
words = set( ramones_midi_data.freq.unique())

#word_indices = dict((c, i) for i, c in enumerate(words))
#indices_word = dict((i, c) for i, c in enumerate(words))

#print("chars:",type(chars))
#print("words",type(words))
print("total number of unique words",len(words))
#print("total number of unique chars", len(chars))

maxlen = 32
step = 1
print("maxlen:",maxlen,"step:", step)

sentences = []
next_words = []
list_words = []

sentences2=[]
list_words = list(ramones_midi_data.freq)

#split the text into sentences of length maxlen --> sentences 
#split the list of words into list of "next_words" 
for i in range(0,len(list_words)-maxlen, step):
    sentences2 = list_words[i: i + maxlen]
    sentences.append(sentences2)
    next_words.append((list_words[i + maxlen]))
    
    
print('nb sequences(length of sentences):', len(sentences))
print("length of next_word",len(next_words))


print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(words)), dtype=np.bool)
y = np.zeros((len(sentences), len(words)), dtype=np.bool)

for ii in range(len(sentences)):
    for jj in range(maxlen):
        #print(i,t,word)
        X[ii, jj, sentences[ii][jj]] = 1
    y[ii, next_words[ii] ] = 1



('total number of unique words', 2344)
('maxlen:', 32, 'step:', 1)
('nb sequences(length of sentences):', 170204)
('length of next_word', 170204)
Vectorization...


In [9]:



#model = Sequential()
#model.add(LSTM(128, return_sequences=True, input_shape=(maxlen, len(words))))
#model.add(Dropout(0.2))
#model.add(LSTM(128, return_sequences=False))
#model.add(Dropout(0.2))
#model.add(Dense(len(words)))
#model.add(Activation('softmax'))
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


#model = Sequential()
#model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
#model.add(Dropout(0.2))
#model.add(Dense(y.shape[1], activation='softmax'))
#model.compile(loss='categorical_crossentropy', optimizer='adam')

model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(maxlen, len(words))))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(words)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


In [10]:
# define the checkpoint
filepath="RAIMONES_v2.0_weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


In [11]:

model.fit(X, y, epochs=50, batch_size=128, callbacks=callbacks_list)

Epoch 1/50


ResourceExhaustedError: OOM when allocating tensor with shape[32,128,256]
	 [[Node: lstm_3/TensorArrayStack/TensorArrayGatherV3 = TensorArrayGatherV3[_class=["loc:@lstm_3/TensorArray"], dtype=DT_FLOAT, element_shape=[?,256], _device="/job:localhost/replica:0/task:0/gpu:0"](lstm_3/TensorArray, lstm_3/TensorArrayStack/range, lstm_3/while/Exit_1)]]
	 [[Node: mul_29/_67 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_3991_mul_29", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'lstm_3/TensorArrayStack/TensorArrayGatherV3', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/ubuntu/.local/lib/python2.7/site-packages/ipykernel/__main__.py", lin